## Label Refinement
This notebook is used for refining labels in manually segmented images (for semantic segmentation). Manual annotation usually causes changes in pixel values near object boundaries. This notebook caters to refine that.

In [1]:
import numpy as np
import cv2
import json
import os

In [2]:
# Specify the dataset directories and the classes file path
data_dir = '/home/jonzamora/Desktop/ARCSeg/src/datasets/miccaiSegOrgans'

if not os.path.exists(data_dir):
    print("ERROR: data_dir" + data_dir + "does not exist!")

# data_dir = '/home/salman/pytorch/segmentationNetworks/datasets/miccaiSeg'
json_path = '/home/jonzamora/Desktop/ARCSeg/src/datasets/classes/miccaiSegClasses.json'

if not os.path.exists(data_dir):
    print("ERROR: json_path" + json_path + "does not exist!")
# json_path = '/home/salman/pytorch/segmentationNetworks/datasets/miccaiSegClasses.json'

# Save directory
save_dir = '/home/jonzamora/Desktop/ARCSeg/src/datasets/miccaiSegRefined'
# save_dir = '/home/salman/pytorch/segmentationNetworks/datasets/miccaiSegRefined'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [3]:
files = {x: [os.path.join(data_dir, x, 'groundtruth', f) for f in os.listdir(os.path.join(data_dir, x, 'groundtruth')) 
         if (f.endswith('.jpg') or f.endswith('.png'))]
         for x in ['train', 'test', 'trainval']}

In [4]:
files

{'train': ['/home/jonzamora/Desktop/ARCSeg/src/datasets/miccaiSegOrgans/train/groundtruth/28200_gt.png',
  '/home/jonzamora/Desktop/ARCSeg/src/datasets/miccaiSegOrgans/train/groundtruth/25775_gt.png',
  '/home/jonzamora/Desktop/ARCSeg/src/datasets/miccaiSegOrgans/train/groundtruth/5025_gt.png',
  '/home/jonzamora/Desktop/ARCSeg/src/datasets/miccaiSegOrgans/train/groundtruth/5900_gt.png',
  '/home/jonzamora/Desktop/ARCSeg/src/datasets/miccaiSegOrgans/train/groundtruth/25250_gt.png',
  '/home/jonzamora/Desktop/ARCSeg/src/datasets/miccaiSegOrgans/train/groundtruth/5400_gt.png',
  '/home/jonzamora/Desktop/ARCSeg/src/datasets/miccaiSegOrgans/train/groundtruth/54400_gt.png',
  '/home/jonzamora/Desktop/ARCSeg/src/datasets/miccaiSegOrgans/train/groundtruth/27025_gt.png',
  '/home/jonzamora/Desktop/ARCSeg/src/datasets/miccaiSegOrgans/train/groundtruth/25450_gt.png',
  '/home/jonzamora/Desktop/ARCSeg/src/datasets/miccaiSegOrgans/train/groundtruth/30675_gt.png',
  '/home/jonzamora/Desktop/ARCSeg/

In [5]:
def disentangleKey(key):
    '''
        Disentangles the key for class and labels obtained from the
        JSON file
        Returns a python dictionary of the form:
            {Class Id: RGB Color Code as numpy array}
    '''
    dKey = {}
    for i in range(len(key)):
        class_id = int(key[i]['id'])
        c = key[i]['color']
        c = c.split(',')
        c0 = int(c[0][1:])
        c1 = int(c[1])
        c2 = int(c[2][:-1])
        color_array = np.asarray([c0,c1,c2])
        dKey[class_id] = color_array

    return dKey

In [6]:
# Get the classes RGB key
classes = json.load(open(json_path))['classes']
key = disentangleKey(classes)

In [7]:
classes

[{'id': '0',
  'name': 'unknown',
  'color': '[170,0,85]',
  'super-category': 'misc'},
 {'id': '1',
  'name': 'grasper',
  'color': '[0,85,170]',
  'super-category': 'instrument'},
 {'id': '2',
  'name': 'bipolar',
  'color': '[0,85,255]',
  'super-category': 'instrument'},
 {'id': '3',
  'name': 'hook',
  'color': '[0,170,85]',
  'super-category': 'instrument'},
 {'id': '4',
  'name': 'scissors',
  'color': '[0,255,85]',
  'super-category': 'instrument'},
 {'id': '5',
  'name': 'clipper',
  'color': '[0,255,170]',
  'super-category': 'instrument'},
 {'id': '6',
  'name': 'irrigator',
  'color': '[85,0,170]',
  'super-category': 'instrument'},
 {'id': '7',
  'name': 'specimen-bag',
  'color': '[85,0,255]',
  'super-category': 'instrument'},
 {'id': '8',
  'name': 'trocars',
  'color': '[170,85,85]',
  'super-category': 'instrument'},
 {'id': '9',
  'name': 'clip',
  'color': '[170,170,170]',
  'super-category': 'instrument'},
 {'id': '10',
  'name': 'liver',
  'color': '[85,170,0]',
 

In [8]:
key

{0: array([170,   0,  85]),
 1: array([  0,  85, 170]),
 2: array([  0,  85, 255]),
 3: array([  0, 170,  85]),
 4: array([  0, 255,  85]),
 5: array([  0, 255, 170]),
 6: array([ 85,   0, 170]),
 7: array([ 85,   0, 255]),
 8: array([170,  85,  85]),
 9: array([170, 170, 170]),
 10: array([ 85, 170,   0]),
 11: array([ 85, 170, 255]),
 12: array([ 85, 255,   0]),
 13: array([ 85, 255, 170]),
 14: array([170,   0, 255]),
 15: array([255,   0, 255]),
 16: array([255, 255,   0]),
 17: array([255,   0,   0]),
 18: array([0, 0, 0])}

In [9]:
# Please run only once, otherwise restart kernel and then run again
for k in range(len(key)):
    rgb = key[k]
    rgb = np.expand_dims(rgb, 0)
    if 'keyMat' in locals():
        keyMat = np.concatenate((keyMat, rgb), axis=0)
    else:
        keyMat = rgb

In [10]:
# Iterate over all images to smooth them
x = ['train', 'test', 'trainval']
for i in range(len(files)):
    folder = files[x[i]]
    save_sub_dir = os.path.join(save_dir, x[i], 'groundtruth')
    if not os.path.exists(save_sub_dir):
        os.makedirs(save_sub_dir)
        
    for j in range(len(folder)):
        img = cv2.imread(folder[j])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        refined = np.zeros_like(img)
        
        # Iterate over all image pixels
        # TODO: Vectorize this
        for h in range(img.shape[0]):
            for w in range(img.shape[1]):
                
                label = np.argmin(np.linalg.norm(np.subtract(img[h,w,:], keyMat), axis = 1))
                rgb = key[label]
                refined[h,w,:] = rgb
        
        # Apply median filtering to remove the salt pepper noise produced at image boundaries
        refined = cv2.medianBlur(refined, 5)
        
        # Save the image
        refined = cv2.cvtColor(refined, cv2.COLOR_BGR2RGB)
        file_name = folder[j].split('/')[-1].split('.')[0] + '.png'
        save_path = os.path.join(save_sub_dir, file_name)
        cv2.imwrite(save_path, refined)
        print('Image: [%d]/[%d]: Folder: [%d]/[%d]' % (j+1, len(folder), i+1, len(files)))

Image: [1]/[245]: Folder: [1]/[3]
Image: [2]/[245]: Folder: [1]/[3]
Image: [3]/[245]: Folder: [1]/[3]
Image: [4]/[245]: Folder: [1]/[3]
Image: [5]/[245]: Folder: [1]/[3]
Image: [6]/[245]: Folder: [1]/[3]
Image: [7]/[245]: Folder: [1]/[3]
Image: [8]/[245]: Folder: [1]/[3]
Image: [9]/[245]: Folder: [1]/[3]
Image: [10]/[245]: Folder: [1]/[3]
Image: [11]/[245]: Folder: [1]/[3]
Image: [12]/[245]: Folder: [1]/[3]
Image: [13]/[245]: Folder: [1]/[3]
Image: [14]/[245]: Folder: [1]/[3]
Image: [15]/[245]: Folder: [1]/[3]
Image: [16]/[245]: Folder: [1]/[3]
Image: [17]/[245]: Folder: [1]/[3]
Image: [18]/[245]: Folder: [1]/[3]
Image: [19]/[245]: Folder: [1]/[3]
Image: [20]/[245]: Folder: [1]/[3]
Image: [21]/[245]: Folder: [1]/[3]
Image: [22]/[245]: Folder: [1]/[3]
Image: [23]/[245]: Folder: [1]/[3]
Image: [24]/[245]: Folder: [1]/[3]
Image: [25]/[245]: Folder: [1]/[3]
Image: [26]/[245]: Folder: [1]/[3]
Image: [27]/[245]: Folder: [1]/[3]
Image: [28]/[245]: Folder: [1]/[3]
Image: [29]/[245]: Folder: [1

Image: [232]/[245]: Folder: [1]/[3]
Image: [233]/[245]: Folder: [1]/[3]
Image: [234]/[245]: Folder: [1]/[3]
Image: [235]/[245]: Folder: [1]/[3]
Image: [236]/[245]: Folder: [1]/[3]
Image: [237]/[245]: Folder: [1]/[3]
Image: [238]/[245]: Folder: [1]/[3]
Image: [239]/[245]: Folder: [1]/[3]
Image: [240]/[245]: Folder: [1]/[3]
Image: [241]/[245]: Folder: [1]/[3]
Image: [242]/[245]: Folder: [1]/[3]
Image: [243]/[245]: Folder: [1]/[3]
Image: [244]/[245]: Folder: [1]/[3]
Image: [245]/[245]: Folder: [1]/[3]
Image: [1]/[62]: Folder: [2]/[3]
Image: [2]/[62]: Folder: [2]/[3]
Image: [3]/[62]: Folder: [2]/[3]
Image: [4]/[62]: Folder: [2]/[3]
Image: [5]/[62]: Folder: [2]/[3]
Image: [6]/[62]: Folder: [2]/[3]
Image: [7]/[62]: Folder: [2]/[3]
Image: [8]/[62]: Folder: [2]/[3]
Image: [9]/[62]: Folder: [2]/[3]
Image: [10]/[62]: Folder: [2]/[3]
Image: [11]/[62]: Folder: [2]/[3]
Image: [12]/[62]: Folder: [2]/[3]
Image: [13]/[62]: Folder: [2]/[3]
Image: [14]/[62]: Folder: [2]/[3]
Image: [15]/[62]: Folder: [2]

Image: [160]/[307]: Folder: [3]/[3]
Image: [161]/[307]: Folder: [3]/[3]
Image: [162]/[307]: Folder: [3]/[3]
Image: [163]/[307]: Folder: [3]/[3]
Image: [164]/[307]: Folder: [3]/[3]
Image: [165]/[307]: Folder: [3]/[3]
Image: [166]/[307]: Folder: [3]/[3]
Image: [167]/[307]: Folder: [3]/[3]
Image: [168]/[307]: Folder: [3]/[3]
Image: [169]/[307]: Folder: [3]/[3]
Image: [170]/[307]: Folder: [3]/[3]
Image: [171]/[307]: Folder: [3]/[3]
Image: [172]/[307]: Folder: [3]/[3]
Image: [173]/[307]: Folder: [3]/[3]
Image: [174]/[307]: Folder: [3]/[3]
Image: [175]/[307]: Folder: [3]/[3]
Image: [176]/[307]: Folder: [3]/[3]
Image: [177]/[307]: Folder: [3]/[3]
Image: [178]/[307]: Folder: [3]/[3]
Image: [179]/[307]: Folder: [3]/[3]
Image: [180]/[307]: Folder: [3]/[3]
Image: [181]/[307]: Folder: [3]/[3]
Image: [182]/[307]: Folder: [3]/[3]
Image: [183]/[307]: Folder: [3]/[3]
Image: [184]/[307]: Folder: [3]/[3]
Image: [185]/[307]: Folder: [3]/[3]
Image: [186]/[307]: Folder: [3]/[3]
Image: [187]/[307]: Folder: 